In [1]:
import tensorflow as tf
import numpy as np
from functions import createGraphRNN_dynamic,createGraphRNN2,RNN_generator_dynamic,RNN_generator_static
import cPickle
from hyperopt import hp, fmin, tpe

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
learningRate2 = 1e-5
epoch = 1
bucketSize = 10000

In [4]:
path = '/home/will/Desktop/Neural Network/Sales Forecasting/val/'

In [5]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [6]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two continous variables

In [7]:
fixedPara = {'cardinalitys_X':cardinalitys_X,'cardinalitys_T':cardinalitys_T,'dimentions_X':dimentions_X\
             ,'dimentions_T':dimentions_T,'dX':dX,'d':d}

In [8]:
prefix = 'train_SS'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [9]:
prefix = 'val_SS'
y_np_val = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np_val = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np_val = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_val = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np_val = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np_val = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [10]:
model_para_list = [{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.628044403851,'n_layers':2,'grad_clip':2.99125114123,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.921508935737},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.813682171767,'n_layers':2,'grad_clip':4.71494036186,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.771180461646},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.702969880055,'n_layers':2,'grad_clip':2.01736824207,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.898455532427},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.428501799036,'n_layers':3,'grad_clip':4.2006247805,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.734974933319},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.459696006212,'n_layers':5,'grad_clip':5.76707667157,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.644960742075},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.427920301113,'n_layers':4,'grad_clip':6.37998154905,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.467438862634},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.716917304571,'n_layers':2,'grad_clip':1.7676565249,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.940803705956},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.677708118391,'n_layers':3,'grad_clip':1.0014287617,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.810976906715},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.801469391508,'n_layers':5,'grad_clip':2.86814589654,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.719331346325},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.819993479712,'n_layers':2,'grad_clip':4.74376042578,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.782327028395},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.415802367601,'n_layers':4,'grad_clip':6.38154110535,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.684181659341},
{'model_para':{'batch_size':100,'seq_len':16,'keep_prob':0.670033419799,'n_layers':2,'grad_clip':1.40163749265,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.82368610599},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.524084618388,'n_layers':3,'grad_clip':7.3201240237,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.520909052689},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.745017883177,'n_layers':2,'grad_clip':7.89519173494,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.902410343017},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.400749504666,'n_layers':4,'grad_clip':4.14237357172,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.913470970128},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.897205518497,'n_layers':3,'grad_clip':4.95813489272,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.583659186288},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.64949104808,'n_layers':2,'grad_clip':2.23405885747,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.670332198251},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.732932282822,'n_layers':2,'grad_clip':1.32638769648,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.694517667059},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.464332365322,'n_layers':2,'grad_clip':4.33557727242,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.941907479812},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.877049685508,'n_layers':3,'grad_clip':6.13619058959,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.884550903377},
]

In [11]:
space = [hp.choice('models',range(len(model_para_list)))] +\
        hp.choice('batch_seq',[[100,16],[50,32],[25,64]]) + \
        [hp.uniform('grad_clip',1,5),\
         'Adam',\
         hp.choice('trainModeParas',[{'trainMode':'dynamic','StopGrad':hp.choice('StopGrad',[True,False])}\
                                    ,{'trainMode':'static'}]),\
         hp.uniform('downsample',0.8,1)]

In [ ]:
def hyperSearch2(paras):   
    # paras[0] is one of the modelName
    model_para = model_para_list[paras[0]]['model_para'].copy()
    model_name = 'grad_clip:{}-cell_type:{}-optimizer:{}-actFun:{}-seq_len:{}-n_layers:{}-keep_prob:{}-batch_size:{}-downsample:{}'\
                 .format(model_para['grad_clip'],model_para['cell_type'],model_para['optimizer'],model_para['actFun'],model_para['seq_len'],\
                         model_para['n_layers'],model_para['keep_prob'],model_para['batch_size'],model_para_list[paras[0]]['downsample']) 
    model_para['batch_size'] = paras[1]
    model_para['seq_len'] = paras[2]
    model_para['grad_clip'] = paras[3]
    model_para['optimizer'] = paras[4] 
    model_para.update(fixedPara)
    trainModeParas = paras[5]
    downsample = paras[6]
    startDate = 0 # set to zero as there is only 64 days
    if trainModeParas['trainMode'] == 'dynamic':
        model_para['StopGrad'] = trainModeParas['StopGrad']
        inputs,train_op,cost,saver,yhat,state = createGraphRNN_dynamic(**model_para)
    else:
        inputs,train_op,cost,saver,yhat,state = createGraphRNN2(**model_para)
        
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,path+model_name)
    
    # Training
    init_state = tuple([tf.contrib.rnn.LSTMStateTuple(np.zeros((model_para['batch_size'],model_para['d']),dtype=np.float32),\
                                                      np.zeros((model_para['batch_size'],model_para['d']),dtype=np.float32))\
                                                    for i in range(model_para['n_layers'])]) \
                 if model_para['cell_type'] == 'NormLSTM' else \
                 tuple([np.zeros((model_para['batch_size'],model_para['d']),dtype=np.float32) \
                        for i in range(model_para['n_layers'])]) 
        
    generator_ = RNN_generator_dynamic if trainModeParas['trainMode'] == 'dynamic' else RNN_generator_static
    for i in range(epoch):
        for X_nps in generator_(y_np, weight_np,Con_np,Dis_np,X_np,\
                                paras[1],paras[2],startDate=startDate,downSample=downsample):
            _,init_state = sess.run([train_op,state],\
                                 dict(zip(inputs,X_nps+[learningRate2,init_state])))
    
    saver.save(sess,'RNN_SS_temp')
        
    # Testing        
    model_para2 = model_para.copy()
    model_para2['batch_size'] = None
    model_para2['seq_len'] = 16
    if trainModeParas['trainMode'] == 'dynamic':
        del model_para2['StopGrad']
    inputs,train_op,cost,saver,yhat,state = createGraphRNN2(**model_para2)   
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'RNN_SS_temp')
    
    ## get init_state
    init_tot_list = []
    for X_nps in RNN_generator_static(y_np, weight_np,Con_np,Dis_np,X_np,\
                                      100,16,startDate=startDate,downSample=1,iterAll=True,permutate=False):
        if X_nps[-1]:
            init_tot_list.append(init_state)
        init_state = sess.run(state,dict(zip(inputs,X_nps+[learningRate2,init_state])))
    init_tot_list.append(init_state)
    init_tot_list = init_tot_list[1:]
    
    # prediction    
    model_para2['StopGrad'] = False # does not matter for prediction
    inputs,train_op,cost,saver,yhat,state = createGraphRNN_dynamic(**model_para2)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'RNN_SS_temp')
    
    loss = 0
    w_ = 0         
    for i,X_nps in enumerate(RNN_generator_dynamic(y_np_val, weight_np_val,Con_np_val,Dis_np_val,X_np_val,\
                                                 100,16,startDate=0,downSample=1,iterAll=True,permutate=False)): 
        X_nps[-2] = False
        loss = loss + sess.run(cost,dict(zip(inputs,X_nps+[learningRate2,init_tot_list[i]])))*X_nps[0].shape[0]*16
        w_ = w_ + np.sum(X_nps[1])
    loss = np.sqrt(loss/w_)
    print "loss:{} , model:{}, trainMode:{}, batch_size:{} ,seq_len:{} ,grad_clip:{} ,downsample:{} ,optimizer:{}  \n"\
          .format(loss,paras[0],trainModeParas['trainMode'],model_para['batch_size'],\
                  model_para['seq_len'],model_para['grad_clip'],downsample,model_para['optimizer'])
    return 100 if (np.isnan(loss) or np.isinf(loss)) else loss     

In [ ]:
best = fmin(hyperSearch2,space,tpe.suggest,60)

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/val/grad_clip:4.33557727242-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.464332365322-batch_size:25-downsample:0.941907479812
INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.67488155866 , model:18, trainMode:dynamic, batch_size:25 ,seq_len:64 ,grad_clip:2.04999564163 ,downsample:0.830262757894 ,optimizer:Adam  

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/val/grad_clip:4.2006247805-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:3-keep_prob:0.428501799036-batch_size:25-downsample:0.734974933319
INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.661653001179 , model:3, trainMode:static, batch_size:100 ,seq_len:16 ,grad_clip:1.75954951587 ,downsample:0.889303138611 

In [27]:
paras

(2,
 100,
 16,
 3.0786340474542677,
 'Adam',
 {'StopGrad': False, 'trainMode': 'dynamic'},
 0.904738282891856)

In [13]:
paras = (2,
 100,
 16,
 3.0786340474542677,
 'Adam',
 {'StopGrad': False, 'trainMode': 'dynamic'},
 0.904738282891856)

In [14]:
hyperSearch2(paras)

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/val/grad_clip:2.01736824207-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.702969880055-batch_size:25-downsample:0.898455532427
INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.676723491543 , model:2, trainMode:dynamic, batch_size:100 ,seq_len:16 ,grad_clip:3.07863404745 ,downsample:0.904738282892 ,optimizer:Adam  

INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.685162569475 , model:2, trainMode:dynamic, batch_size:100 ,seq_len:16 ,grad_clip:3.07863404745 ,downsample:0.904738282892 ,optimizer:Adam  

INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.689833815795 , model:2, trainMode:dynamic, batch_size:100 ,seq_len:16 ,grad_clip:3.07863404745 ,downsample:0.904738282

In [17]:
hyperSearch2(paras)

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/val/grad_clip:2.01736824207-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.702969880055-batch_size:25-downsample:0.898455532427
INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.669431731015 , model:2, trainMode:dynamic, batch_size:100 ,seq_len:16 ,grad_clip:3.07863404745 ,downsample:0.904738282892 ,optimizer:Adam  



In [20]:
hyperSearch2(paras)

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/val/grad_clip:2.01736824207-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.702969880055-batch_size:25-downsample:0.898455532427
INFO:tensorflow:Restoring parameters from RNN_SS_temp
INFO:tensorflow:Restoring parameters from RNN_SS_temp
loss:0.675821872063 , model:2, trainMode:dynamic, batch_size:100 ,seq_len:16 ,grad_clip:3.07863404745 ,downsample:0.904738282892 ,optimizer:Adam  

